In [1]:
from __future__ import print_function, division, unicode_literals
%load_ext autoreload
%autoreload 2
import math
import time
import os
import copy
import unicodedata
import string
import re
import random
import io
import json
from io import open

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import distributions as pyd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib import cm
%matplotlib inline
import seaborn as sns
from PIL import Image
import cv2
from tqdm import tqdm

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
plt.ion()   # interactive mode - o

import vima_bench
from vima_bench import PARTITION_TO_SPECS

/Users/andipeng/miniforge3/envs/align-lang/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pybullet build time: Apr 20 2023 11:39:48


[INFO] 17 tasks loaded


In [2]:
# Custom weight init for Conv2D and Linear layers
def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.orthogonal_(m.weight.data)
        if hasattr(m.bias, "data"):
            m.bias.data.fill_(0.0)
                
# helper mlp init function
def mlp(input_dim, output_dim, hidden_dim, hidden_depth, output_mod=None):
    if hidden_depth == 0:
        mods = [nn.Linear(input_dim, output_dim)]
    else:
        mods = [nn.Linear(input_dim, hidden_dim), nn.ReLU(inplace=True)]
        for i in range(hidden_depth - 1):
            mods += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU(inplace=True)]
        mods.append(nn.Linear(hidden_dim, output_dim))
    if output_mod is not None:
        mods.append(output_mod)
    trunk = nn.Sequential(*mods)
    return trunk

class Flatten(nn.Module):
    def forward(self, x):
        return x.reshape(x.size(0), -1)

# MLP policy
class CNNPolicy(nn.Module):
    def __init__(
        self, action_dim, hidden_size, output_mod=None):
        super().__init__()
        self.cnntrunk = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4), nn.ReLU(inplace=True), nn.BatchNorm2d(32), #(b_size,3,36,72)=>(b_size,32,8,17)
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(inplace=True), nn.BatchNorm2d(64), #(b_size,32,8,17)=>(b_size,64,3,7)
            nn.Conv2d(64, 32, kernel_size=3, stride=1), nn.LeakyReLU(inplace=True), nn.BatchNorm2d(32), Flatten(), #(b_size,64,3,7)=>(b_size,32,1,5)=>(b_size,32*1*5)
            nn.Linear(32*1*5, action_dim)#, nn.LeakyReLU(inplace=True), nn.BatchNorm1d(32), #(b_size,32*1*5)=>(b_size,action_dim)
        )
        self.apply(weight_init)

    def forward(self, state):
        state = state/255.0 # process image + switch channels
        state = state.permute(0,3,1,2)
        
        pred = self.cnntrunk(state)
        return pred

In [3]:
def downsize_obs(obs):
    cv2.imwrite('temp.jpg', obs)
    img = cv2.imread('temp.jpg')
    downsized_obs = cv2.resize(img, dsize=(72, 36), interpolation=cv2.INTER_CUBIC)
    return downsized_obs

def flatten_act(action):
    return np.concatenate(list(action.values())).ravel()

# reconstructs actions for simulator
def reconstruct_act(action):
    reconst_action = {}
    reconst_action['pose0_position'] = np.array(action[0:2])
    reconst_action['pose0_rotation'] = np.array(action[2:6])
    reconst_action['pose1_position'] = np.array(action[6:8])
    reconst_action['pose1_rotation'] = np.array(action[8:12])
    reconst_action = {
        k: np.clip(v, env.action_space[k].low, env.action_space[k].high)
        for k, v in reconst_action.items()
    }
    return reconst_action

def remove_obj(segm, obs, remove_obj):
    #if remove_obj == 'arm':
    #    segm = (segm == 2).astype(int)
    if remove_obj == 'base':
        segm = (segm == 5).astype(int)
    elif remove_obj == 'dragged':
        segm = (segm == 6).astype(int)
    elif remove_obj == 'distractor':
        segm = (segm == 7).astype(int)
    segm = np.atleast_3d(segm)
    
    for height in range(128):
        for width in range(256):
            if segm[height, width] == 1:
                obs[height, width] = 47
    return obs

def compare_actions(actions, gt_action):
    comparisons = []
    for action in actions:
        comparisons.append(np.linalg.norm(gt_action - action[[0,1,6,7]],ord=2))
    return comparisons

# generates random trajs within specified constraints
def gen_trajs(env, num_trajs, task_name, task_kwargs):
    trajs = []
    task = env.task
    oracle_fn = task.oracle(env)
    for traj in tqdm(range(num_trajs)):
        traj = {'obs': [],'acts': [], 'meta': []}
        obs = env.reset()
        traj['meta'] = env.meta_info
        for step in range(1):
            top_obs = obs['rgb']['top'] # extracts top down view only
            top_obs = np.rollaxis(top_obs,0,3)
            traj['obs'].append(downsize_obs(top_obs.copy()))
            # prompt, prompt_assets = env.prompt, env.prompt_assets
            oracle_action = oracle_fn.act(obs)
            # clip action
            oracle_action = {
                k: np.clip(v, env.action_space[k].low, env.action_space[k].high)
                for k, v in oracle_action.items()
            }
            traj['acts'].append(flatten_act(oracle_action))
            obs, _, done, info = env.step(action=oracle_action, skip_oracle=False)
        traj['obs'] = np.array(traj['obs'])
        traj['acts'] = np.array(traj['acts'])
        traj['meta'] = np.array(traj['meta'])
        trajs.append(traj)
    return trajs

In [ ]:
task_name = 'visual_manipulation'
#task_kwargs=PARTITION_TO_SPECS["train"][task_name]
task_kwargs = { 'num_dragged_obj': 1,
     'num_base_obj': 1,
     'num_other_obj': 0,
     'dragged_obj_loc': [3],
     'base_obj_loc': [1],
     'third_obj_loc' : [2],
     'fourth_obj_loc' : [4],
     'possible_dragged_obj': ['flower'],
     'possible_dragged_obj_texture': ['green'],
     'possible_base_obj': ['bowl'],
     'possible_base_obj_texture': ['yellow'],
     'possible_third_obj': ['letter A'],
     'possible_third_obj_texture': ['purple'],
     'possible_fourth_obj': ['pentagon'],
    'possible_fourth_obj_texture': ['blue']}
#record_gui=True, display_debug_window=True, hide_arm_rgb=False
env = vima_bench.make(task_name=task_name,task_kwargs=task_kwargs,hide_arm_rgb=False)

num_trajs = 5
trajs = gen_trajs(env=env, num_trajs=num_trajs, task_name=task_name, task_kwargs=task_kwargs)
pickle.dump(trajs, open('trajs.pkl', 'wb'))
env.close()
#obs = env.reset()
#top_obs = obs['rgb']['top']
#top_obs = np.rollaxis(top_obs,0,3)
#plt.imshow(top_obs)
#env.close()

In [ ]:
#trajs = pickle.load(open('trajs.pkl','rb'))
plt.imshow(trajs[0]['obs'][0])

In [ ]:
num_tasks = len(trajs)

act_size = trajs[0]['acts'][0].shape[0]
hidden_size = 100

policy = CNNPolicy(act_size, hidden_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
policy.to(device)

In [ ]:
num_epochs = 500
batch_size = 10

criterion = nn.MSELoss()
optimizer = optim.Adam(list(policy.parameters()))

losses = []

idxs = np.array(range(len(trajs)))

num_batches = len(idxs) // batch_size
# Train the model with regular SGD
for epoch in range(num_epochs):  # loop over the dataset multiple times
    np.random.shuffle(idxs)
    running_loss = 0.0
    for i in range(num_batches):
        optimizer.zero_grad()

        t_idx = np.random.randint(len(trajs), size=(batch_size,)) # Indices of traj
        t_idx_pertraj = np.random.randint(1, size=(batch_size,)) # Indices of timesteps in traj
        t_states = np.concatenate([trajs[c_idx]['obs'][t_idx][None] for (c_idx, t_idx) in zip(t_idx, t_idx_pertraj)])
        t_actions = np.concatenate([trajs[c_idx]['acts'][t_idx][None] for (c_idx, t_idx) in zip(t_idx, t_idx_pertraj)])
   
        t_states = torch.Tensor(t_states).float().to(device)
        t_actions = torch.Tensor(t_actions).float().to(device)
        
        a_preds = policy(t_states)[0]
        loss = torch.mean(torch.linalg.norm(a_preds - t_actions, dim=-1)) # supervised learning loss
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.8f' %
                  (epoch + 1, i + 1, running_loss))
            losses.append(running_loss)
            running_loss = 0.0
        losses.append(loss.item())

torch.save(policy, 'policy.pt')
print('Finished Training')
plt.plot(losses)

In [23]:
task_name = "rearrange"
#task_kwargs = { 'num_dragged_obj': 1,
#     'num_base_obj': 1,
#     'num_other_obj': 0,
#     'dragged_obj_loc': [3],
#     'base_obj_loc': [1],
#     'third_obj_loc' : [2],
#     'fourth_obj_loc' : [4],
#     'possible_dragged_obj': ['flower'],
#     'possible_dragged_obj_texture': ['green'],
#     'possible_base_obj': ['bowl'],
#     'possible_base_obj_texture': ['yellow'],
#     'possible_third_obj': ['letter A'],
#     'possible_third_obj_texture': ['purple'],
#     'possible_fourth_obj': ['pentagon'],
#     'possible_fourth_obj_texture': ['blue']}
#task_kwargs = {'possible_dragged_obj': ['pentagon','flower']}
record_cfg = {'save_video': True,
     'save_video_path': './rollouts/',
     'view': 'front',
     'fps': 20,
     'video_height': 640,
     'video_width': 720}
# record_gui=True, display_debug_window=True, hide_arm_rgb=False
env = vima_bench.make(task_name=task_name,task_kwargs=None,hide_arm_rgb=False,record_cfg=record_cfg)
#obs = env.reset()
#segm = obs['segm']['top']
#top_obs = obs['rgb']['top']
#top_obs = np.rollaxis(top_obs,0,3)
#plt.imshow(remove_obj(segm, top_obs, 'dragged'))

In [ ]:
oracle = True

if oracle:
    task = env.task
    oracle_fn = task.oracle(env)
else:
    policy = torch.load('policy.pt')
    policy.eval()

obj_to_remove = None#['base']
num_test_trajs = 1
video = True

plt.xlim(-0.5, 0.5)
plt.ylim(0.3, 0.7)

successes = []

rollouts = {'actions': [], 'action_starts': [],'action_ends': [], 'true_starts': [], 'true_ends': []}

for i in tqdm(range(num_test_trajs)):
    os.makedirs('rollouts/' + str(i), exist_ok=True)
    obs = env.reset()
    #rollouts['true_starts'].append(env.task.dragged_pose[0])
    #rollouts['true_ends'].append(env.task.base_pose[0])
    
    if video:
        video_name = str(i)
        env.start_rec(video_name)
    for step in range(1):
        if oracle:
            oracle_action = oracle_fn.act(obs)
            # clip action
            oracle_action = {
                k: np.clip(v, env.action_space[k].low, env.action_space[k].high)
                for k, v in oracle_action.items()
            }
            obs, _, done, info = env.step(action=oracle_action, skip_oracle=False)
        
        else:
            segm = obs['segm']['top']
            top_obs = obs['rgb']['top']
            top_obs = np.rollaxis(top_obs,0,3)

            if obj_to_remove is not None:
                obj_removed = random.choice(obj_to_remove)
                segm = obs['segm']['top']
                top_obs = remove_obj(segm, top_obs, obj_removed)
            first_obs = top_obs.copy()

            im = Image.fromarray(top_obs)
            im.save('rollouts/'+str(i)+"/"+str(step)+'.jpg')
            top_obs = downsize_obs(top_obs)
            state = torch.Tensor(top_obs[None]).to(device)
            action = policy(state).cpu().detach().numpy()[0]
            
            rollouts['actions'].append(action)
            rollouts['action_starts'].append(action[0:2].copy())
            rollouts['action_ends'].append(action[6:8].copy())
            obs, _, done, info = env.step(action=reconstruct_act(action), skip_oracle=False)
        
        if done:
            successes.append(1)
        else:
            successes.append(0)
            
    if video:
        env.end_rec()
        
    top_obs = obs['rgb']['top']
    top_obs = np.rollaxis(top_obs,0,3)
    
    if obj_to_remove is not None:
        segm = obs['segm']['top']
        top_obs = remove_obj(segm, top_obs, obj_removed)
        
    im = Image.fromarray(top_obs)
    im.save('rollouts/'+str(i)+"/"+str(step+1)+'.jpg')

env.close()

if not oracle:
    plt.scatter(np.array(rollouts['true_starts'])[:, 1], np.array(rollouts['true_starts'])[:, 0], marker='o', color='g', label='gt starts')
    plt.scatter(np.array(rollouts['true_ends'])[:, 1], np.array(rollouts['true_ends'])[:, 0], marker='x', color='g', label='gt ends')
    plt.scatter(np.array(rollouts['action_starts'])[:, 1], np.array(rollouts['action_starts'])[:, 0], marker='o', color='r', label='rollout starts')
    plt.scatter(np.array(rollouts['action_ends'])[:, 1], np.array(rollouts['action_ends'])[:, 0], marker='x', color='r', label='rollout ends')
    plt.gca().invert_yaxis()
    plt.legend()
print(sum(successes)/len(successes))

  0%|                                                                          | 0/1 [00:00<?, ?it/s]

In [ ]:
plt.imshow(first_obs)

In [ ]:
plt.imshow(top_obs)